In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/heart.csv")
df.dropna(inplace=True)
display(df.describe().iloc[[0,3,7]])
display(pd.DataFrame.from_dict(dict(zip(df.columns.tolist(), [[df[x].nunique()] for x in df.columns.tolist()]))))
df.to_csv (r'data/heart.csv', index = False, header=True)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.0,3656.00,3656.0,3656.0,3656.0
min,0.0,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,113.0,83.5,48.0,15.54,44.0,40.0,0.0
max,1.0,70.0,4.0,1.0,70.0,1.0,1.0,1.0,1.0,600.0,295.0,142.5,56.80,143.0,394.0,1.0


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,2,39,4,2,33,2,2,2,2,241,231,142,1297,72,138,2


In [3]:
import pandas_profiling
from pandas_profiling import ProfileReport

In [4]:
report = df.profile_report()
report.to_file(output_file="results/przed_preprocessingiem.html")

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

W wygenerowanym raporcie widać zależność sysBP od diaBP.
Używamy VIF do sprawdzenia tego.

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
X = df[['diaBP', 'sysBP', 'diabetes', 'education', 'glucose', 'currentSmoker']]
    
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data)

         feature         VIF
0          diaBP  106.772663
1          sysBP   99.486567
2       diabetes    1.515865
3      education    4.460419
4        glucose   16.475212
5  currentSmoker    1.897497


Widać, że jest problem

Sprawdzamy też korelacje.

In [6]:
target = "TenYearCHD"
numFeatures = ['age', 'totChol', 'cigsPerDay', 'diaBP', 'sysBP', 'BMI', 'heartRate', 'glucose']
catFeatures = [ 'male', 'education', 'currentSmoker',  'BPMeds', 'prevalentStroke', 'prevalentHyp',  'diabetes']

In [7]:
from sklearn import preprocessing

In [8]:
df_o = pd.read_csv("data/heart.csv")
df = df_o.copy()
df = df_o.copy()
mapy = {}
for feature in catFeatures:
    # Inicjujemy obiekt do kodowania, który będzie przechowywal naszą mapę
    le = preprocessing.LabelEncoder()
    # Dopasowujemy dane (tworzymy mapę i od razu ją nakładamy na zmienną)
    df[feature] = le.fit_transform(df_o[feature])
    # Zapiszmy mapę, aby móc odzyskać oryginalne dane bez problemu
    mapy[feature] = le

In [9]:
from scipy import stats
from sklearn import feature_selection

In [10]:
df[target] = (df_o[target] == 1).astype(int)

In [12]:
minfos=[]
for var in catFeatures:
    print("\n", var)
    print("Kendall", stats.kendalltau(df[target], df[var]))
    print("Mutual info", feature_selection.mutual_info_classif(df[var].values.reshape(-1,1),df[target].values))
    minfos.append(feature_selection.mutual_info_classif(df[var].values.reshape(-1,1),df[target].values)[0])
    print("Chi2", feature_selection.chi2(df[var].values.reshape(-1,1),df[target].values))
    print("Anova", feature_selection.f_classif(df[var].values.reshape(-1,1),df[target].values))

imp = list(zip(minfos, catFeatures))
imp.sort(reverse=True)
imp


 male
Kendall KendalltauResult(correlation=0.09174488519402911, pvalue=2.913056101400075e-08)
Mutual info [0.00562216]
Chi2 (array([17.12043013]), array([3.50828854e-05]))
Anova (array([31.01724696]), array([2.74186569e-08]))

 education
Kendall KendalltauResult(correlation=-0.06906201810736914, pvalue=6.58928523901043e-06)
Mutual info [0.00694934]
Chi2 (array([15.51824879]), array([8.17125909e-05]))
Anova (array([14.59196477]), array([0.0001357]))

 currentSmoker
Kendall KendalltauResult(correlation=0.019176196282060395, pvalue=0.24632269545786645)
Mutual info [0.00028407]
Chi2 (array([0.68691311]), array([0.40721634]))
Anova (array([1.34416693]), array([0.24637621]))

 BPMeds
Kendall KendalltauResult(correlation=0.08911570359021277, pvalue=7.139200453828643e-08)
Mutual info [0.00274691]
Chi2 (array([28.15300258]), array([1.12093636e-07]))
Anova (array([29.25093742]), array([6.76635126e-08]))

 prevalentStroke
Kendall KendalltauResult(correlation=0.04835057303345267, pvalue=0.0034655

[(0.01077116375969922, 'prevalentHyp'),
 (0.005600311244622569, 'diabetes'),
 (0.004697321202663973, 'currentSmoker'),
 (0.0021544237087762763, 'education'),
 (0.0007966033301918252, 'prevalentStroke'),
 (4.228190243349417e-05, 'BPMeds'),
 (0, 'male')]

In [13]:
for var in numFeatures:
    print("Values for pair {0}".format(var))
    print("Spearman", stats.spearmanr(df[target], df[var]))
    print("Mutual info", feature_selection.mutual_info_classif(df[var].values.reshape(-1,1),df[target].values))
    minfos.append(feature_selection.mutual_info_classif(df[var].values.reshape(-1,1),df[target].values)[0])

imp = list(zip(minfos, catFeatures+numFeatures))
imp.sort(reverse=True)
imp

Values for pair age
Spearman SpearmanrResult(correlation=0.2308387525955526, pvalue=2.025044957476851e-45)
Mutual info [0.03373422]
Values for pair totChol
Spearman SpearmanrResult(correlation=0.08597336437171171, pvalue=1.9271397056260287e-07)
Mutual info [0.00687696]
Values for pair cigsPerDay
Spearman SpearmanrResult(correlation=0.03934446939402349, pvalue=0.017356582684077852)
Mutual info [0.00572341]
Values for pair diaBP
Spearman SpearmanrResult(correlation=0.13192470192572078, pvalue=1.1554254425155279e-15)
Mutual info [0.01537164]
Values for pair sysBP
Spearman SpearmanrResult(correlation=0.1976648363586101, pvalue=1.5790946698340526e-33)
Mutual info [0.01817213]
Values for pair BMI
Spearman SpearmanrResult(correlation=0.0794010072857738, pvalue=1.5327328240018309e-06)
Mutual info [0.00711086]
Values for pair heartRate
Spearman SpearmanrResult(correlation=0.013800615879396145, pvalue=0.40416448048334586)
Mutual info [0.00087074]
Values for pair glucose
Spearman SpearmanrResult(

[(0.02865262166747895, 'age'),
 (0.020098871149548225, 'sysBP'),
 (0.01077116375969922, 'prevalentHyp'),
 (0.005733860673471813, 'BMI'),
 (0.005600311244622569, 'diabetes'),
 (0.004697321202663973, 'currentSmoker'),
 (0.004439122081159486, 'diaBP'),
 (0.002282457616612943, 'cigsPerDay'),
 (0.0021544237087762763, 'education'),
 (0.0020563178083898848, 'totChol'),
 (0.00137420525392562, 'heartRate'),
 (0.0007966033301918252, 'prevalentStroke'),
 (4.228190243349417e-05, 'BPMeds'),
 (0, 'male'),
 (0, 'glucose')]

sysBP ważniejszą zmienną od diaBP (zgadza się to z wiedzą medyczna)

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm

# Przygotujmy prosty wrapper
def CVTest(nFolds = 5, randomState=2020, features=[]):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # Aby oszczędzać pamięć informacja o foldach to wyłącznie numery wierszy
    testResults = []
    for train, test in kf.split(df.index.values):
        mod = sm.GLM.from_formula(formula=target + " ~ "+ "+".join(features),
                                  data=df_o.iloc[train], family=sm.families.Binomial())
        res = mod.fit()
        predsTrain = res.predict(df_o.iloc[train])
        preds = res.predict(df_o.iloc[test])
        testResults.append(roc_auc_score(df.TenYearCHD.iloc[test], 1-preds))
        
    return np.mean(testResults)

In [15]:
imps2 = []
for var in numFeatures+catFeatures:
    res = CVTest(features=[var])
    imps2.append((res, var))
    print(var, res)

age 0.31468446591254573
totChol 0.43201436458847003
cigsPerDay 0.4706062917507271
diaBP 0.39438070988875495
sysBP 0.3413852090395781
BMI 0.43629397462758523
heartRate 0.4894312868523114
glucose 0.4557795348571517
male 0.4362242421025263
education 0.4430714515943735
currentSmoker 0.486907353365796
BPMeds 0.4789445234163038
prevalentStroke 0.494715726909544
prevalentHyp 0.38295930690108687
diabetes 0.47891404341722604


In [16]:
imps2.sort(reverse=True)
imps2

[(0.494715726909544, 'prevalentStroke'),
 (0.4894312868523114, 'heartRate'),
 (0.486907353365796, 'currentSmoker'),
 (0.4789445234163038, 'BPMeds'),
 (0.47891404341722604, 'diabetes'),
 (0.4706062917507271, 'cigsPerDay'),
 (0.4557795348571517, 'glucose'),
 (0.4430714515943735, 'education'),
 (0.43629397462758523, 'BMI'),
 (0.4362242421025263, 'male'),
 (0.43201436458847003, 'totChol'),
 (0.39438070988875495, 'diaBP'),
 (0.38295930690108687, 'prevalentHyp'),
 (0.3413852090395781, 'sysBP'),
 (0.31468446591254573, 'age')]

Tutaj diaBP i sysBP podobne. Usuwamy diaBP.

Usuwamy też cigsPerDay i logarytmujemy niektóre zmienne.

Usuwamy choroby (0-1) ze względu niezbalansowanie. Nie łączymy chorób w jedną zmienną ponieważ nie wykazaliśmy korelacji pomiędzy chorobami i nie chcemy tracić interpretacji zmiennej prevalentHyp (nadciśnienie) na zmienną objaśnianą.

In [17]:
df = pd.read_csv("data/heart.csv")

df = df.drop(columns=['cigsPerDay', 'diaBP', 'BPMeds', 'diabetes', 'prevalentStroke'])

df['log_glucose'] = np.log(df['glucose'])
df = df.drop(columns=['glucose'])

df['log_BMI'] = np.log(df['BMI'])
df = df.drop(columns=['BMI'])

df['log_sysBP'] = np.log(df['sysBP'])
df = df.drop(columns=['sysBP'])

df['log_totChol'] = np.log(df['totChol'])
df = df.drop(columns=['totChol'])

df.to_csv (r'data/ver2.csv', index = False, header=True)

In [18]:
report = df.profile_report()
report.to_file(output_file="results/po_preprocessingu_ver2.html")

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

---

Poniżej inna propozycja modyfikacji zbioru danych (usuniecie niezbalansowanych zmiennych zero-jedynkowych i 'ogonów')

In [19]:
df = pd.read_csv("data/heart.csv")

df = df.drop(columns=['cigsPerDay', 'diaBP', 'BPMeds', 'diabetes', 'prevalentStroke'])

df = df[df.glucose < 200]

df['log_glucose'] = np.log(df['glucose'])
df = df.drop(columns=['glucose'])

df = df[df.totChol < 400]

df = df[df.sysBP < 225]

df = df[df.BMI < 45]

df = df[df.heartRate < 130]

df['log_BMI'] = np.log(df['BMI'])
df = df.drop(columns=['BMI'])

df['log_sysBP'] = np.log(df['sysBP'])
df = df.drop(columns=['sysBP'])

df['log_totChol'] = np.log(df['totChol'])
df = df.drop(columns=['totChol'])

df.to_csv (r'data/ver3.csv', index = False, header=True)

In [20]:
report = df.profile_report()
report.to_file(output_file="results/po_preprocessingu_ver3.html")

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]